<a href="https://colab.research.google.com/github/ancka019/ComputationsMethods6sem/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
from copy import copy
from numpy.linalg import norm
from scipy.linalg import hilbert, eig

In [2]:
def max_abs(a): #максимальный по модулю в матрице элемент
    i_max,j_max = 0,1
    max_item = a[i_max,j_max]
    for i in range(a.shape[0]):
        for j in range(i+1, a.shape[0]):
            if abs(max_item) < abs(a[i,j]):
                max_item = a[i, j]
                i_max, j_max = i, j
    return i_max, j_max

In [3]:
def jacobi_method(a,eps,strategy="circle"): #метод Якоби 
        iters = 0
        i,j = 0,0
        while True:
            h = np.identity(a.shape[0])
            if strategy == "abs":
                i,j = max_abs(a)
            else:
                if (j < (a.shape[0]-1) and j+1!=i):
                    j+=1
                elif j == a.shape[0]-1:
                    i+=1
                    j = 0
                else:
                    j+=2
            if i==a.shape[0]-1 and j==a.shape[0]:
                    return np.diag(a), iters
            if abs(a[i, j]) < eps:
                return np.diag(a), iters
            iters += 1
            phi = 0.5*(math.atan((2*a[i, j])/(a[i,i]-a[j,j])))
            c,s = math.cos(phi), math.sin(phi)
            h[i,i], h[j,j] = c,c
            h[i,j], h[j,i] = -s, s
            a = h.T@a@h

In [4]:
def gersh_circles(a): #определение кругов Гершгорина
    ans = []
    for i in range(a.shape[0]):
        ans.append((a[i,i],sum(abs(a[i]))-abs(a[i,i])))
    return ans


def is_in_circle(gersh,lmda): #проверка в принадлежности с.ч. хотя бы одному кругу
    return any([abs(c-lmda)<=r for c,r in gersh])

In [6]:
X0 = np.array([[-5.509882,1.870086,0.422908],
              [0.287865,-11.811654,5.7119],
              [0.049099,4.308033,-12.970687]]) #матрица из учебника Н.В. Фаддевой и Д.К. Фаддеева

matrixes = [X0,*[hilbert(n) for n in range(3,6)],hilbert(20)]

In [7]:
X = pd.DataFrame(columns=['eps=10^(-2),res','eps=10^(-2),iters',
                          'eps=10^(-3),res', 'eps=10^(-3),iters',
                          'eps=10^(-4),res','eps=10^(-4),iters',
                         'eps=10^(-5),res','eps=10^(-5),iters'])

Y = pd.DataFrame(columns=['eps=10^(-2),res','eps=10^(-2),iters',
                          'eps=10^(-3),res', 'eps=10^(-3),iters',
                          'eps=10^(-4),res','eps=10^(-4),iters',
                         'eps=10^(-5),res','eps=10^(-5),iters'])

for matrix in matrixes:
    lambda_true = np.sort(eig(matrix)[0])
    row_X,row_Y = [],[]
    for i in range(2,6):
        lambda_abs,abs_iters = jacobi_method(matrix,10**(-i),strategy="abs")
        lambda_circle,circle_iters = jacobi_method(matrix,10**(-i),strategy="circle")
        row_X.extend([norm(np.sort(lambda_abs)-lambda_true),abs_iters])
        row_Y.extend([norm(np.sort(lambda_circle)-lambda_true),circle_iters])
    X = X.append(pd.Series(row_X,index=X.columns),True)
    Y = Y.append(pd.Series(row_Y,index=Y.columns),True)
    
X.index = ['X0','hilbert(3)','hilbert(4)','hilbert(5)','hilbert(20)']
Y.index = ['X0','hilbert(3)','hilbert(4)','hilbert(5)','hilbert(20)']

#стратетия с максимальным по модулю с.ч

In [8]:
X

,"eps=10^(-2),res","eps=10^(-2),iters","eps=10^(-3),res","eps=10^(-3),iters","eps=10^(-4),res","eps=10^(-4),iters","eps=10^(-5),res","eps=10^(-5),iters"
X0,7.574484e-04,7.0,7.828313e-04,8.0,1.797801e-07,10.0,1.797801e-07,10.0
hilbert(3),3.070874e-07,5.0,3.070874e-07,5.0,2.846794e-12,6.0,2.846794e-12,6.0
hilbert(4),3.665365e-03,8.0,7.597559e-08,11.0,7.597559e-08,11.0,3.023919e-13,15.0
hilbert(5),8.019429e-03,11.0,1.008748e-04,17.0,2.186581e-07,18.0,3.994225e-11,24.0
hilbert(20),7.265768e-02,55.0,7.767278e-04,107.0,4.174993e-05,160.0,5.488484e-06,212.0


#стретия обнуления по порядку

In [9]:
Y

,"eps=10^(-2),res","eps=10^(-2),iters","eps=10^(-3),res","eps=10^(-3),iters","eps=10^(-4),res","eps=10^(-4),iters","eps=10^(-5),res","eps=10^(-5),iters"
X0,2.953740e-01,6.0,2.953740e-01,6.0,2.953740e-01,6.0,2.953740e-01,6.0
hilbert(3),7.479139e-07,4.0,7.479139e-07,4.0,4.298829e-07,5.0,4.298829e-07,5.0
hilbert(4),3.664689e-03,7.0,1.045717e-04,10.0,1.045717e-04,10.0,1.045840e-04,11.0
hilbert(5),8.004709e-03,9.0,6.619628e-03,11.0,4.321995e-04,19.0,4.322053e-04,20.0
hilbert(20),7.388318e-02,40.0,7.301494e-02,41.0,7.301595e-02,42.0,7.301596e-02,44.0


#Проверка принадлежность найденных значений кругам Гершорина

In [10]:
for matrix in matrixes:
    lambda_abs = jacobi_method(matrix,10**(-5),strategy="abs")[0]
    gersh = gersh_circles(matrix)
    print(all(([is_in_circle(gersh,lmbd) for lmbd in lambda_abs])))

True
True
True
True
True
